<a href="https://colab.research.google.com/github/adhyaanilkumar/voice-cloner/blob/adhya-branch/01_environment_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


Wed Sep  3 07:51:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -q numpy scipy librosa matplotlib unidecode inflect tqdm tensorboard


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.9 MB/s eta 0:00:00


In [ ]:
import torch, torchaudio
print("torch", torch.__version__, "cuda?", torch.cuda.is_available())


torch 2.8.0+cu126 cuda? True


In [ ]:
!git clone https://github.com/NVIDIA/tacotron2
!git clone https://github.com/NVIDIA/waveglow


Cloning into 'tacotron2'...
remote: Enumerating objects: 412, done.
remote: Total 412 (delta 0), reused 0 (delta 0), pack-reused 412 (from 1)
Receiving objects: 100% (412/412), 2.70 MiB | 7.20 MiB/s, done.
Resolving deltas: 100% (203/203), done.
Cloning into 'waveglow'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 196 (delta 2), reused 2 (delta 0), pack-reused 190 (from 1)
Receiving objects: 100% (196/196), 437.57 KiB | 1.44 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [ ]:
import torch
t2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub','nvidia_tacotron2',pretrained=True).eval()
wg = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub','nvidia_waveglow',pretrained=True)
wg = wg.remove_weightnorm(wg)
print("Loaded Tacotron2 & WaveGlow.")


/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip


/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded Tacotron2 & WaveGlow.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

BASE = "/content/drive/MyDrive/Colab Notebooks/voice-cloner"  # <- your folder
RAW_ROOT   = f"{BASE}/data/raw"
CLEAN_ROOT = f"{BASE}/data/clean"

import os
os.makedirs(RAW_ROOT, exist_ok=True)
os.makedirs(CLEAN_ROOT, exist_ok=True)

%cd "$BASE"
!pwd && ls -la


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/voice-cloner
/content/drive/MyDrive/Colab Notebooks/voice-cloner
total 27
-rw------- 1 root root 8998 Sep  6 17:02 01_environment_setup.ipynb
drwx------ 4 root root 4096 Sep  6 17:02 data
drwx------ 2 root root 4096 Sep  6 16:39 .git
-rw------- 1 root root 4895 Sep  5 12:52 .gitignore
-rw------- 1 root root 1092 Sep  5 12:52 LICENSE
-rw------- 1 root root 2581 Sep  5 12:52 README.md


In [11]:
import os, glob, numpy as np
import librosa, soundfile as sf
from tqdm import tqdm

TARGET_SR = 22050
TOP_DB = 20
PEAK = 0.95

def process_one(in_wav, out_wav):
    y, sr = librosa.load(in_wav, sr=None, mono=True)
    if sr != TARGET_SR:
        y = librosa.resample(y, orig_sr=sr, target_sr=TARGET_SR)
    y, _ = librosa.effects.trim(y, top_db=TOP_DB)
    if y.size == 0:
        return False
    m = np.max(np.abs(y))
    if m > 0:
        y = y / m * PEAK
    os.makedirs(os.path.dirname(out_wav), exist_ok=True)
    sf.write(out_wav, y, TARGET_SR, subtype="PCM_16")
    return True

def mirror_clean(raw_root, clean_root):
    print(f"mirror_clean: Searching in raw_root: {raw_root}") # Added print
    files = glob.glob(f"{raw_root}/**/*.wav", recursive=True)
    print("Found", len(files), "files")
    if len(files) > 0: # Added print for sample files
        print("Sample raw files found:", files[:5])
    ok, skip = 0, 0
    for f in tqdm(files, desc="Standardizing"):
        rel = os.path.relpath(f, raw_root)
        out = os.path.join(clean_root, os.path.splitext(rel)[0] + ".wav")
        if ok < 5: # Added print for sample output paths
            print(f"Processing {f} -> Saving to {out}")
        try:
            if process_one(f, out):
                ok += 1
            else:
                skip += 1
        except Exception as e:
            skip += 1
            print("Error:", f, e)
    print(f"Done. Wrote {ok}, skipped {skip}.")

print(f"Calling mirror_clean with raw_root: {RAW_ROOT}")
mirror_clean(RAW_ROOT, CLEAN_ROOT)

Calling mirror_clean with raw_root: /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw
mirror_clean: Searching in raw_root: /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw
Found 2638 files
Sample raw files found: ['/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0184.wav', '/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0186.wav', '/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0187.wav', '/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0192.wav', '/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0183.wav']


Standardizing:   0%|          | 4/2638 [00:00<01:24, 31.22it/s]

Processing /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0184.wav -> Saving to /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/LJSpeech-1.1/wavs/LJ004-0184.wav
Processing /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0186.wav -> Saving to /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/LJSpeech-1.1/wavs/LJ004-0186.wav
Processing /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0187.wav -> Saving to /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/LJSpeech-1.1/wavs/LJ004-0187.wav
Processing /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0192.wav -> Saving to /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/LJSpeech-1.1/wavs/LJ004-0192.wav
Processing /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/wavs/LJ004-0183.wav -> Saving to /content/drive/MyDrive/Col

Standardizing: 100%|██████████| 2638/2638 [01:18<00:00, 33.44it/s]

Done. Wrote 2638, skipped 0.


In [12]:
import os, csv, glob

EMO_MAP = {
    "Amused":"amused", "Angry":"angry", "Disgusted":"disgusted",
    "Neutral":"neutral", "Sleepy":"sleepy"
}

MANIFEST = f"{BASE}/manifests"
os.makedirs(MANIFEST, exist_ok=True)
MANIFEST_CSV = f"{MANIFEST}/manifest.csv"

rows = []

# LJSpeech
lj_clean = f"{CLEAN_ROOT}/LJSpeech-1.1"
lj_raw_meta = f"{RAW_ROOT}/LJSpeech-1.1/metadata.csv"
print(f"Checking for LJSpeech metadata at: {lj_raw_meta}") # Added print
if os.path.exists(lj_raw_meta):
    print("LJSpeech metadata found. Processing LJSpeech data.") # Added print
    with open(lj_raw_meta, "r", encoding="utf-8") as f:
        rdr = csv.reader(f, delimiter="|")
        for wid, text, norm in rdr:
            wav = f"{lj_clean}/wavs/{wid}.wav"
            if os.path.exists(wav):
                rows.append([wav, (norm or text).strip(), "lj", "neutral", "train"])
else:
    print(f"LJSpeech metadata not found at {lj_raw_meta}. Skipping LJSpeech processing.") # Added print


# Emotions
def sidecar_text(path_wo_ext):
    for ext in (".txt",".lab",".trs"):
        p = path_wo_ext + ext
        if os.path.exists(p):
            try:
                return open(p, "r", encoding="utf-8").read().strip()
            except:
                pass
    return ""

emo_root = f"{CLEAN_ROOT}/emotion"
print(f"Checking for emotion data in clean directory: {emo_root}") # Added print
if os.path.isdir(emo_root):
    print("Clean emotion directory found. Processing emotion data.") # Added print
    for speaker in sorted(os.listdir(emo_root)):
        sp_dir = f"{emo_root}/{speaker}"
        if not os.path.isdir(sp_dir):
            continue
        for emo_folder in sorted(os.listdir(sp_dir)):
            emo = EMO_MAP.get(emo_folder, None)
            if emo is None:
                continue
            wavs = sorted(glob.glob(f"{sp_dir}/{emo_folder}/**/*.wav", recursive=True))
            print(f"Found {len(wavs)} emotion wav files in {sp_dir}/{emo_folder}") # Added print
            for i, w in enumerate(wavs):
                text = sidecar_text(os.path.splitext(w)[0])
                if not text: # Added print if text is empty
                    print(f"Warning: No sidecar text found for {w}. Skipping or using empty text.") # Decide how to handle empty text
                split = "val" if i % 20 == 0 else "train"
                rows.append([w, text, speaker, emo, split])
else:
    print(f"Clean emotion directory not found at {emo_root}. Skipping emotion processing.") # Added print

print(f"Total rows collected for manifest: {len(rows)}") # Added print

with open(MANIFEST_CSV, "w", encoding="utf-8", newline="") as f:
    w = csv.writer(f)
    w.writerow(["path","text","speaker","emotion","split"])
    w.writerows(rows)

len(rows), MANIFEST_CSV

Checking for LJSpeech metadata at: /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/metadata.csv
LJSpeech metadata not found at /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw/LJSpeech-1.1/metadata.csv. Skipping LJSpeech processing.
Checking for emotion data in clean directory: /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/emotion
Clean emotion directory not found at /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/emotion. Skipping emotion processing.
Total rows collected for manifest: 0


(0,
 '/content/drive/MyDrive/Colab Notebooks/voice-cloner/manifests/manifest.csv')

In [14]:
import glob, os

print("Raw root:", RAW_ROOT)
print("Clean root:", CLEAN_ROOT)

# Check how many files were standardized
clean_wavs = glob.glob(f"{CLEAN_ROOT}/**/*.wav", recursive=True)
print("Number of clean wavs:", len(clean_wavs))
print("First few:", clean_wavs[:5])

# Check if LJSpeech metadata exists
print("LJSpeech metadata present?", os.path.exists(f"{RAW_ROOT}/LJSpeech-1.1/metadata.csv"))


Raw root: /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/raw
Clean root: /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean
Number of clean wavs: 2638
First few: ['/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/LJSpeech-1.1/wavs/LJ004-0184.wav', '/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/LJSpeech-1.1/wavs/LJ004-0186.wav', '/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/LJSpeech-1.1/wavs/LJ004-0187.wav', '/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/LJSpeech-1.1/wavs/LJ004-0192.wav', '/content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/LJSpeech-1.1/wavs/LJ004-0183.wav']
LJSpeech metadata present? False


In [13]:
import os, glob

print("Checking for files in emotion directory:")
emotion_wavs = glob.glob(f"{CLEAN_ROOT}/emotion/**/*.wav", recursive=True)
print(f"Found {len(emotion_wavs)} wav files in {CLEAN_ROOT}/emotion")
if len(emotion_wavs) > 0:
    print("First 5 emotion wav files:", emotion_wavs[:5])
    print("\nChecking sidecar text for the first emotion file:")
    first_emotion_wav_path = emotion_wavs[0]
    first_emotion_text = sidecar_text(os.path.splitext(first_emotion_wav_path)[0])
    print(f"Text for {first_emotion_wav_path}: '{first_emotion_text}'")
else:
    print("No emotion wav files found. Please check if the emotion dataset is correctly placed in the raw data directory and the cleaning process ran successfully.")

Checking for files in emotion directory:
Found 0 wav files in /content/drive/MyDrive/Colab Notebooks/voice-cloner/data/clean/emotion
No emotion wav files found. Please check if the emotion dataset is correctly placed in the raw data directory and the cleaning process ran successfully.
